In [15]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

dataset_path = "Dataset"
data = []

for painter in os.listdir(dataset_path):
    painter_folder = os.path.join(dataset_path, painter)

    if not os.path.isdir(painter_folder) or painter.startswith('.'):
        continue

    for img_file in os.listdir(painter_folder):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(painter_folder, img_file)
            data.append({'filepath': img_path, 'painter': painter})

df = pd.DataFrame(data)

le = LabelEncoder()
df['label'] = le.fit_transform(df['painter'])

label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print("🎨 Label mapping:")
for k, v in label_map.items():
    print(f"{k}: {v}")

df.to_csv("image_labels.csv", index=False)
print("CSV 文件已保存为 image_labels.csv")

🎨 Label mapping:
Ai_Xuan_艾軒: 0
Chang_Dai_Chien_張大千: 1
Chu_Chu_儲楚: 2
Giuseppe_Castiglione_郎世宁: 3
Gu_Wenda_谷文达: 4
He_Baili_何百里: 5
He_Jilan_何紀嵐: 6
Jin_Ting_Biao_金廷标: 7
Li_Fang_Ying_李方膺: 8
Li_Zijian_李自健: 9
Ling_Jian_凌健: 10
Liu_Kuo_Sung_刘国松: 11
Liu_Yuan_Shou_劉元壽: 12
Lu_Xin_Jian_陆新建: 13
Luo_Jian_Wu_罗建武: 14
Luo_Zhong_Li_羅中立: 15
Qin_Feng_秦风: 16
Qiu_Ying_仇英: 17
Wai_Ming: 18
Wang_Hui_王翚: 19
Yao_Wen_Han_姚文瀚: 20
Yu_Zhi_Ding_禹之鼎: 21
Zhang_Xiao_Gang_张晓刚: 22
Zhao_Kailin_赵开霖: 23
Zou_Chuan_An_邹传安: 24
CSV 文件已保存为 image_labels.csv


In [16]:
import os
import pandas as pd
from PIL import Image
import torch
from torchvision import transforms

IMAGE_SIZE = (224, 224)

transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

df = pd.read_csv("image_labels.csv")

tensor_dir = "image_tensors"
os.makedirs(tensor_dir, exist_ok=True)

for idx, row in df.iterrows():
    img_path = row['filepath']
    painter = row['painter']
    label = row['label']

    try:
        image = Image.open(img_path).convert("RGB")
        tensor = transform(image)  # [3, 224, 224]

        img_name = os.path.splitext(os.path.basename(img_path))[0]
        save_path = os.path.join(tensor_dir, f"{img_name}.pt")

        torch.save({'tensor': tensor, 'label': label}, save_path)

        if idx % 100 == 0:
            print(f"✅ Saved: {save_path}")

    except Exception as e:
        print(f"❌ Error loading {img_path}: {e}")

print("所有图像已成功转换为张量并保存")

✅ Saved: image_tensors/Jin_Ting_Biao_08.pt
✅ Saved: image_tensors/Lu_Xin_Jian_14.pt
✅ Saved: image_tensors/Qin_Feng_17.pt
✅ Saved: image_tensors/He_Jilan_02.pt
所有图像已成功转换为张量并保存


In [18]:
success_count = 0
fail_count = 0

for idx, row in df.iterrows():
    img_path = row['filepath']
    painter = row['painter']
    label = row['label']

    try:
        image = Image.open(img_path).convert("RGB")
        tensor = transform(image)

        img_name = os.path.splitext(os.path.basename(img_path))[0]
        save_path = os.path.join(tensor_dir, f"{img_name}.pt")

        torch.save({'tensor': tensor, 'label': label}, save_path)
        success_count += 1

        if idx % 100 == 0:
            print(f"✅ Saved: {save_path}")

    except Exception as e:
        print(f"❌ Error loading {img_path}: {e}")
        fail_count += 1

print(f" 处理完成，共成功保存 {success_count} 张，失败 {fail_count} 张。")

✅ Saved: image_tensors/Jin_Ting_Biao_08.pt
✅ Saved: image_tensors/Lu_Xin_Jian_14.pt
✅ Saved: image_tensors/Qin_Feng_17.pt
✅ Saved: image_tensors/He_Jilan_02.pt
 处理完成，共成功保存 396 张，失败 0 张。


In [17]:
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

pt_path = "image_tensors/Qin_Feng_06.pt"

data = torch.load(pt_path)

tensor = data['tensor'] 
label = data['label']       

print("shape of tensor:", tensor.shape)
print("number of label:", label)

shape of tensor: torch.Size([3, 224, 224])
number of label: 16


/var/folders/7s/nq14qz0x5fbgzmf2n7c9_w0r0000gn/T/ipykernel_17565/435760075.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_path)
